In [68]:
import pandas as pd
from keras.layers import Input, Dense, LSTM, Embedding, RepeatVector
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [69]:
data_path = "../data/processed/dataset.csv"
data = pd.read_csv(data_path)
data = data.sample(n=1000, random_state=42)

In [70]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Lyric'].astype(str))
sequences = tokenizer.texts_to_sequences(data['Lyric'].astype(str))
max_sequence_length = max(len(seq) for seq in sequences)
sequences_padded = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
total_words = len(tokenizer.word_index) + 1

In [71]:
X_train, X_test = train_test_split(sequences_padded, test_size=0.2, random_state=42)

In [72]:
def create_autoencoder_model(max_sequence_length, total_words):
    inputs = Input(shape=(max_sequence_length,))
    encoded = Embedding(total_words, 128, input_length=max_sequence_length)(inputs)
    encoded = LSTM(128)(encoded)

    decoded = RepeatVector(max_sequence_length)(encoded)
    decoded = LSTM(128, return_sequences=True)(decoded)
    decoded = Dense(total_words, activation='softmax')(decoded)

    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return autoencoder

In [73]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
autoencoder_model = create_autoencoder_model(max_sequence_length, total_words)

In [74]:
autoencoder_model.fit(X_train, X_train,
                epochs=50,
                batch_size=32,
                validation_data=(X_test, X_test),
                callbacks=[early_stopping])

Epoch 1/50


25/25 [==============================] - 121s 5s/step - loss: 7.3967 - accuracy: 0.6979 - val_loss: 4.5396 - val_accuracy: 0.7093
Epoch 2/50
25/25 [==============================] - 110s 4s/step - loss: 2.9501 - accuracy: 0.7274 - val_loss: 2.5713 - val_accuracy: 0.7093
Epoch 3/50
25/25 [==============================] - 111s 4s/step - loss: 2.3656 - accuracy: 0.7274 - val_loss: 2.5020 - val_accuracy: 0.7093
Epoch 4/50
25/25 [==============================] - 110s 4s/step - loss: 2.3277 - accuracy: 0.7274 - val_loss: 2.5030 - val_accuracy: 0.7093
Epoch 5/50
25/25 [==============================] - 110s 4s/step - loss: 2.3245 - accuracy: 0.7274 - val_loss: 2.5042 - val_accuracy: 0.7093
Epoch 6/50
25/25 [==============================] - 110s 4s/step - loss: 2.3171 - accuracy: 0.7274 - val_loss: 2.4901 - val_accuracy: 0.7093
Epoch 7/50
25/25 [==============================] - 110s 4s/step - loss: 2.2963 - accuracy: 0.7274 - val_loss: 2.4676 - val_accuracy: 0.7093
Epoch 8/50
25/25 [======

In [77]:
autoencoder_model.save("autoencoder_model.h5")

C:\Users\Jurica\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [78]:
def generate_poem(seed_text, model, tokenizer, max_sequence_length, num_words_to_generate):
    generated_text = seed_text
    for _ in range(num_words_to_generate):
        seed_sequence = tokenizer.texts_to_sequences([generated_text])
        padded_sequence = pad_sequences(seed_sequence, maxlen=max_sequence_length, padding='post')
        encoded_text = model.predict(padded_sequence)
        decoded_sequence = tokenizer.sequences_to_texts(encoded_text.argmax(axis=-1))
        generated_text = " ".join(decoded_sequence)
    return generated_text


In [79]:
seed_word = 'blue ocean'
generated_poem = generate_poem(seed_word, autoencoder_model, tokenizer, max_sequence_length, 10)

# Ispisivanje generirane pjesme
print(f"Pjesma generirana na temelju riječi '{seed_word}':\n{generated_poem}")

1/1 [==============================] - 0s 274ms/step
Pjesma generirana na temelju riječi 'blue ocean':

